In this notebook, we will share our methodology in predicting the XXRunCoT and XXRunLang.

We will utilize the API from OpenRouter for the model Qwen2-72B.

For the dataset, we will utilize the data from COLIEE Dataset H30, R01, and R02 (in total 216 entries)

You can download the data from here:

https://drive.google.com/file/d/100LqHADN6P-ROQTyiHMq4op1C2z_icfY/view?usp=sharing

## XXRunCoT

In this run, we employs three different prompt, including chain of thought prompting, graph of thought prompting, and tree of thought prompting

In [ ]:
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import os

df = pd.read_csv('COLIEE_dataset_H30-R02.csv')
df.drop('same_translate', axis=1, inplace=True)
df['cot_result'] = None
df['tot_result'] = None
df['got_result'] = None

In [ ]:
df.head(1)

,file_name,id,label,t1_japan,t2_japan,t1,t2,cot_result,tot_result,got_result
0,riteval_H30_jp.xml,H30-11-O,Y,第二百四十八条 第二百四十二条から前条までの規定の適用によって損失を受けた者は、第七百三条及...,Ａの所有する液体甲（１００立方メートル）が，Ｂの所有する液体乙（１０立方メートル）と混和して...,Article 248\nA person that suffers loss becaus...,In cases liquid P (100 cubic meters) owned by ...,None,None,None


In [ ]:
df.shape

(216, 10)

In [ ]:
#@title Prompt Template for CoT, ToT, and GoT

# Define the prompts for CoT, ToT, and GoT
cot_prompt_template = """
You are a legal expert analyzing legal entailment. Follow these steps:
1. Identify the core legal principle from the given article.
2. Compare it with the statement to determine if it is explicitly supported, implicitly supported, or contradicted.
3. Justify your reasoning with reference to legal logic.
4. Provide a final entailment decision on a **confidence scale (1-5)**.

---

Legal Article:
{t1_japan}

Statement:
{t2_japan}

- [Answer]

Step 2: Compare the principle with the statement:
- [Does the statement match the article? If yes, how? If no, why?]

Step 3: Provide a final entailment decision using the confidence scale below:
1 - Highly confident it is contradicted
2 - Shows slight contradiction
3 - Neutral (neither entailed nor contradicted)
4 - Shows slight entailment
5 - Highly confident it is entailed

- [Final answer: Select a number from 1 to 5 based on the level of entailment]"

At the end of your response, state the answer in single number, either 1 or 2 or 3 or 4 or 5.
"""

tot_prompt_template = """
You are a legal expert analyzing legal entailment using a structured thought process. Follow these steps:
1. Identify the core legal principle from the given article.
2. Branch into different reasoning paths based on possible interpretations:
   - Path A: If the legal principle is clear and directly supports or contradicts the statement.
   - Path B: If the principle is ambiguous, analyze multiple possible interpretations.
   - Path C: If external legal precedents or logic influence the conclusion.
3. For each path, determine the entailment level:
   - Explicitly supported
   - Implicitly supported
   - Contradicted
   - Neutral
4. Compare and weigh the strongest argument.
5. Decide the final entailment decision on a confidence scale (1-5).

You can add reasoning first. But, at the end of your response, state the answer in single number, either 1 or 2 or 3 or 4 or 5.
---

Legal Article:
{t1_japan}

Statement:
{t2_japan}

- [Answer]

Step 2: Explore reasoning paths:
- Path A (Direct Support/Contradiction): [Does the statement clearly follow the principle?]
- Path B (Ambiguity): [Are there multiple interpretations?]
- Path C (Legal Precedent): [Would external legal logic affect the meaning?]

Step 3: Compare and weigh the arguments:
- [Which path is the most convincing? Justify.]

Step 4: Final entailment decision using the confidence scale below:
1 - Highly confident it is contradicted
2 - Shows slight contradiction
3 - Neutral (neither entailed nor contradicted)
4 - Shows slight entailment
5 - Highly confident it is entailed

- [Final answer: Select a number from 1 to 5 based on the strongest path]"

You can add reasoning first. But, at the end of your response, state the answer in single number, either 1 or 2 or 3 or 4 or 5.
"""

got_prompt_template = """
You are a legal expert analyzing legal entailment using a dynamic reasoning graph. Follow these steps:
1. Identify the legal principle from the article.
2. Construct a reasoning graph where:
   - Each node represents a reasoning step.
   - Edges represent logical connections between steps.
   - Cycles indicate dependencies, such as legal precedent influencing interpretation.
3. Explore multiple reasoning paths:
   - Explicit entailment (direct support)
   - Implicit entailment (contextual reasoning)
   - Contradiction (logical/legal mismatch)
   - Neutrality (ambiguity or lack of evidence)
4. Weigh graph connections to assess argument strength:
   - Strong connections indicate direct and well-supported arguments.
   - Weak connections indicate speculative or indirect reasoning.
5. Determine the final entailment decision based on graph strength and coherence.

---

Legal Article:
{t1_japan}

Statement:
{t2_japan}

- [Answer]

Step 2: Construct the reasoning graph:
- Node 1: [Key legal rule/concept]
- Node 2 → Direct Support? [Yes/No]
- Node 3 → Implicit Interpretation? [Yes/No]
- Node 4 → External Legal Precedent? [Yes/No]

Step 3: Weigh the graph’s logical connections:
- [Identify strongest logical paths & contradictions]

Step 4: Final entailment decision using the confidence scale below:
1 - Highly confident it is contradicted
2 - Shows slight contradiction
3 - Neutral (neither entailed nor contradicted)
4 - Shows slight entailment
5 - Highly confident it is entailed

- [Final answer: Select a number from 1 to 5 based on the strongest path in the reasoning graph]"

You can add reasoning first. But, at the end of your response, state the answer in single number, either 1 or 2 or 3 or 4 or 5.
"""


In this method, we utilize the Qwen 2-72B LLM from OpenRouter, you need to create your own API Key to try on this code

In [ ]:
# Initialize the OpenAI client with OpenRouter
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="<YOUR_API_KEY>",
)

# Define a function to get the response using OpenRouter API
def get_entailment_decision(article, statement, prompt_template):
    prompt = prompt_template.format(t1_japan=article, t2_japan=statement)
    try:
        completion = client.chat.completions.create(
            extra_headers={
                "HTTP-Referer": "<YOUR_SITE_URL>",
                "X-Title": "<YOUR_SITE_NAME>",
            },
            model="qwen/qwen-2-72b-instruct",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        print(completion.choices[0].message.content.strip())
        return completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error: {e}")
        return "Error"

This code is dedicated to forward the prompted statements to the LLM. If there are entries that aren't evaluated, it will only re-iterate the entries that haven't been predicted

In [ ]:
tqdm.pandas()
# Check which rows are unprocessed and apply prompts only to them
def process_rows(df, column_name, prompt_template):
    """
    Processes rows where column_name is empty or NaN.
    """
    mask = df[column_name].isnull() | (df[column_name] == "Error") | (df[column_name] == "")
    if mask.sum() == 0:
        print(f"✅ All rows for {column_name} are already processed!")
        return df
    print(f"⚡ Processing {mask.sum()} rows for {column_name}...")

    df.loc[mask, column_name] = df[mask].progress_apply(
        lambda row: get_entailment_decision(row['t1_japan'], row['t2_japan'], prompt_template), axis=1
    )

    return df

In [ ]:
# Prediction using chain of thought prompting
df = process_rows(df, 'cot_result', cot_prompt_template)

⚡ Processing 216 rows for cot_result...


  1%|          | 2/216 [00:07<13:33,  3.80s/it]

Step 1: The core legal principles identified in this legal article can be summarized as follows:
- Article 248: A person who has suffered a loss due to the application of the provisions from Article 242 to the preceding article can request compensation according to the provisions of Articles 703 and 704.
- Article 245: The provisions of the preceding two articles (which talk about mixed properties where the owners differ and the property cannot be identified) apply by analogy to cases where properties owned by different parties have become inseparable due to mixing.
- Article 243: When several movable properties (chattels) owned by different people have been so admixed that separation without damage is impossible, the ownership of the resultant material belongs to the owner of the predominant property; the same applies where the cost of separation would overshadow the value left.
- Article 703: A person who has derived 'unjust enrichment' or 'undue profit' at the cost of another's prop

  1%|▏         | 3/216 [00:11<13:02,  3.67s/it]

Step 1: The core legal principle of the given article is that when it's impossible to distinguish between primary and secondary properties in a conjoined movable property, the property is shared between the respective owners according to the price ratio of their properties at the time of the conjoining.

Step 2: The statement describes a scenario wherein ship owned by A, which is valued at 6 million yen, is conjoined with an engine owned by B, valued at 4 million yen. It further explains that if these properties cannot be distinguished as principal and accessory, and cannot be separated without damage, the engine-equipped ship should be shared by A and B in the ratio of 3:2 according to the respective values of their property. In this context, the statement matches the principle.

Step 3: According to legal logic, since the statement reproduces the principle described in the article in a specific context, it is highly confident that the statement is entailed by the article.

- [Final A

...

Step 1: The core legal principle identified from the given article is that the possessor (in this case, the rightful owner) who has had their possession (an object) taken away can file a claim for recovery of possession (for return of the item) and seek damages. However, such a claim cannot be made against the specific inheritor of the person who initially took the possession, except when the inheritor was aware of the fact that the possession was taken forcibly.

Step 2: Comparing the principle with the given statement, it is indicated that A's personal computer with an owner name label was stolen by B. Later, B transferred the computer, without modifying its exterior appearance, to C who was oblivious to the theft. The statement then suggests that A cannot demand the return of the PC from C via a possession recovery suit.

Step 3: From a legal standpoint, the statement does not exactly contradict Article 200, given that this article explicitly states that the right to file a possessi

100%|██████████| 216/216 [18:00<00:00,  5.20s/it]

Step 1: Identify the core legal principle from the given article.
The core legal principle from the given article (Article 201) is that a "suit for preservation of possession must be filed within one year from the existence of the hindrance or its disappearance. However, in cases where damage to the possessed object occurs due to construction, if one year has passed since the commencement of the work, or if the construction is completed, such a suit cannot be filed."

Step 2: Compare the principle with the statement:
The statement presents a situation in which A owns a plot of land on which B, without permission, starts building a wall that crosses over into A's land. The wall took two years to complete. The statement posits that A cannot invoke the "suit for preservation of possession" against B to have the wall removed at the point of its completion. This is aligned with the exception described in the legal article, which states that such a suit cannot be brought after the constructi

100%|██████████| 216/216 [18:08<00:00,  5.04s/it]

Step 1: Identify the core legal principle from the article.
The core legal principle stated in the article is that an occupant (possessor) whose possession has been taken from them has the legal right to seek recovery through a legal suit. Specifically, this right can be exercised to request the return of the object and compensation for damages as well.

Step 2: Compare the principle with the statement.
The statement describes a scenario where A leases a machine to B, which is subsequently stolen by C during the lease period. Despite not being the original possessor (A is), B is positioned to independently initiate a legal action for possession recovery against C. The key connection of the legal article to this statement is that it endorses the legal right to seek recovery (possession and damages) from the party that has taken their possession forcefully. However, the statement goes beyond this by implying that a lessee (B) has the same rights as the owner (A), which may not always be 

In [ ]:
# Prediction using tree of thought prompting
df = process_rows(df, 'tot_result', cot_prompt_template)

⚡ Processing 216 rows for tot_result...


  1%|          | 2/216 [00:08<14:18,  4.01s/it]

Step 1: Identify the core legal principle from the given article:
The core legal principle in the given article focuses on several aspects:
- Article 248 allows for compensation clauses under certain conditions, referring back to Articles 703 and 704.
- Article 245 deals with the applicability of rules of compensation when properties of different owners become non-distinguishably mixed.
- Article 243 discusses the ownership of synthesized items when properties are amalgamated to the point where separation is no longer possible, especially in the case of real property, assigning it to the owner of the principal property; it also addresses the scenario where separation would entail excessive expense.
- Article 703 deals with unjust enrichment, where someone benefits without a legal reason to the detriment of another, obliging them to return the profit within the limits of its existence.
- Article 704 stipulates that malicious beneficiaries must return the benefit received with interest, 

  1%|▏         | 3/216 [00:11<13:22,  3.77s/it]

Step 1: The core legal principle from the given article is that when it is impossible to distinguish between principal and accessory in combined movable properties, the owners should share the combined property in proportion to the value of their respective properties at the time of combination.

Step 2: The statement matches this principle. In the statement, property A (a ship valued at 6 million yen) and property B (an engine valued at 4 million yen) are combined and cannot be separated without damage. Thus, they cannot distinguish the principal from the accessory. The statement concludes that property A and B should be shared in a 3:2 ratio (3 parts for A and 2 parts for B), which is consistent with their respective values at the time of combination.

Step 3: According to legal logic, since the statement follows the principle stated in the legal article and the decision is derived logically from that principle, it is a proper application of the law.

Final entailment decision: 5

Th

...

Step 1: The core legal principle from the given legal article is that "an occupant who has been deprived of their possession may bring an action for recovery of possession, requesting the return of the item and compensation for damages. However, such an action cannot be filed against the specific successors of the person who wrongfully deprived the possession, unless such successors were aware of the act of deprivation."

Step 2: The statement says, "A, the owner and possessor of a PC marked with an 'Owner A' sticker, had it stolen by B. Later, B transferred the PC without making any outer appearance changes to C, who was unaware of the theft. In such an instance, A cannot file an action for possession recovery, to request the return of the PC from C."

The statement implicitly contradicts the principle by neglecting to mention the awareness of C, the successor, regarding the wrongful deprivation. If C was unaware, the article suggests that A cannot file an action for possession recove

100%|██████████| 216/216 [17:47<00:00,  4.81s/it]

Step 1: The core legal principle from the given article (Japan Civil Code Article 201) relates to the statute of limitations for claims of possession maintenance. A lawsuit for possession maintenance can be filed within one year of the end of the interference or its last moment, but following the start or the completion of construction that led to damage to a possession, no claim can be made.

Step 2: The statement involves a situation where A, who owns a vacation home lot, has an adjacent lot owned by B, who began building a wall on A’s property without permission. After two years, the wall is completed. The statement claims A cannot seek the removal of the wall through a possession maintenance lawsuit at this point.

Step 3: This is entailed by the article because the article specifies that no claim can be made once the construction is completed. Two years have passed from the start of the construction, and the wall is now finished. This aligns with the condition to prevent a possess

100%|██████████| 216/216 [17:55<00:00,  4.98s/it]

Step 1: Identify the core legal principle from the given article.
The core legal principle in the legal article is that a possessor (占有者) who has been dispossessed has the legal right to file a suit for recovery of possession (占有回収の訴え) and can request the return of the object (物の返還) as well as compensation for damages (損害の賠償).

Step 2: Compare the principle with the statement:
The statement discusses a situation where A leases (賃貸) a machine to B, and subsequently, the machine is stolen (窃取) by C while still under the lease terms with B. The statement suggests that B can independently file a suit for recovery of possession against C.

In the context of Japanese law, the concept of '占有' (possession) covers both rightful owners and persons with lawful possession (such as a lessee). Therefore, "Ｂは，Ａから独立して，Ｃに対して占有回収の訴えを提起することができる" (B can independently sue C for recovery of possession) aligns with the legal principle that a possessor can file a suit for recovery and damages even if they are

In [ ]:
# Prediction using graph of thought prompting
df = process_rows(df, 'got_result', cot_prompt_template)

⚡ Processing 216 rows for got_result...


  1%|          | 2/216 [00:06<11:38,  3.26s/it]

Step 1: The core legal principles derived from the legal article are:
- A person who suffers a loss due to the application of Article 242 to Article 247 can claim compensation under Articles 703 and 704.
- When properties owned by different parties become irreversibly mixed and cannot be identified separately, the provisions in the previous two articles apply.
- When multiple movable properties, owned by different parties, are attached together in a way that disengagement would cause disproportionate damage or cost, the property right to the composite item belongs to the main property's owner. The same applies if separation requires disproportionate expenses.

Step 2: The statement considers a situation where A's liquid “甲” and B's liquid “乙” have become mixed and resulted in a new unseparable liquid “丙”. If A ends up owning the all of “丙”, B can claim recompense from A under the principle of unjust enrichment, applying Articles 703 and 704. The statement aligns with Article 245 – whic

  1%|▏         | 3/216 [00:10<12:49,  3.61s/it]

Step 1: The core legal principle from the legal article (Article 244) is that when it's impossible to distinguish between the primary and secondary items in the case of indistinguishable attached movables, each owner of the items must share the compound item according to the value at the time of attachment.

Step 2: The statement gives a situation where a ship owned by A (worth 6 million yen) is fitted with an engine owned by B (worth 4 million yen), and these items are indistinguishable and cannot be separated without damage. According to the legal principle, A and B would have to share the engine-equipped ship in the ratio of their individual items’ values at the time of fitting (6:4 or simplified to 3:2).

Step 3: The statement directly applies and aligns with the legal principle provided in Article 244, as it refers to the same situation of an indistinguishable attached property where the property cannot be separated without damage, and the sharing is proposed based on the value at

...

Step 1: The core legal principle from the given article is that a possessor who has been deprived of their possession can file an action for recovery of possession to demand the return of the property and compensation for damages. However, this action cannot be brought against a specific successor of the person who usurped the possession, unless the successor was aware of the usurpation.

Step 2: The principle implicitly supports the statement. In the given statement, A had their property stolen by B, who then transferred the property to C, who was unaware of the theft. According to the legal principle, A cannot bring an action for recovery of possession against C since C is a successor of B, and was unaware of the theft at the time of transfer. This means that A cannot request the return of the laptop from C via an action for recovery of possession.

Step 3: Legal logic supports that one's right to pursue an action for recovery of possession is barred when the property is transferred 

100%|██████████| 216/216 [18:03<00:00,  4.50s/it]

Step 1: The core legal principle from the given article is that a possessory interdict (占有保持の訴え) can only be initiated within one year after the occurrence of a disturbance or within one year after the completion of any construction that has resulted in damage to the possessed item. There is also an exception stated that if a year has passed since the beginning of the construction or its completion, such a lawsuit cannot be initiated.

Step 2: Comparing this principle with the statement, we see that A owns a piece of land and B, without A's permission, starts building a wall that encroaches upon A's land. The wall is completed two years after the start of construction. The principle stipulates that A would not be able to file a possessory interdict to demolish the wall once a year has passed since the beginning of the construction or its completion. Since B completed the wall two years after the start of the construction which is beyond the one-year limit stipulated by law, the stateme

100%|██████████| 216/216 [18:11<00:00,  5.05s/it]

Step 1: Identify the core legal principle.
The core legal principle from the article is that if someone's possession is taken away from them, they can file a lawsuit for the recovery of that possession and claim damage compensation (Article 200).

Step 2: Compare the principle with the statement.
In the statement, it is mentioned that A, who leased a machine tool to B, had it stolen by C during the lease period. It is then stated that B can independently file a lawsuit against C for the recovery of possession. However, the statement does not clarify if B had possession at the time of theft.

The principle in the Article is that the original possessor has the right to recovery. In the given scenario, it was A who had the original possession and B merely was holding the work machine as a lessee. Therefore, the statement implies B can act independently while in reality, considering the legal principle, it might be A who has the direct right to recover possession and seek damages.

Step 3:

The code below is the processing step to extract the label from the LLM response

In [ ]:
import re

# Define a function to extract the entailment decision from the end of the response
def extract_entailment_number(response):
    # Reverse the response and search for the first number between 1-5 from the end
    reversed_response = response[::-1]  # Reverse the string
    match = re.search(r'[1-5]', reversed_response)  # Look for a digit between 1-5
    if match:
        # Get the matched number and reverse it back to the original order
        return int(match.group(0))  # Extract the matched number
    else:
        return None  # Return None if no valid number is found

# Apply the extraction function to each result column
df['cot_score'] = df['cot_result'].apply(extract_entailment_number)
df['tot_score'] = df['tot_result'].apply(extract_entailment_number)
df['got_score'] = df['got_result'].apply(extract_entailment_number)

In [ ]:
#@title Prompt Template for Neutral Case
# Neutral prompt for cases where the score is 3
neutral_prompt_template = """
You are a legal expert evaluating a claim in a complex legal case in Japan. The claim does not have a clear Yes or No answer because its validity depends on specific conditions (Condition A supports Yes; Condition B supports No).

Legal Article:
{t1_japan}

Statement:
{t2_japan}

Your task is to:
1. Identify the key legal factors that make the case neutral.
2. Determine whether precedent, burden of proof, or judicial tendencies slightly favor one side.
3. If there is no decisive ruling, slightly favor the position that aligns more with established legal reasoning.
4. Provide a structured explanation of your reasoning.

Format Your Answer:

* Legal Factors for Neutrality: (Explain why the case is neutral)
* Japanese Legal Interpretation: (Which statutory rule, business custom, or precedent applies?)
* Potential Leaning: (Does the case slightly favor entailment or contradiction?)
* Final Decision with Justification: ("While Neutral, based on [factor], it leans slightly toward Entailment/Contradiction.")

You can add reasoning first. But, at the end of your response, state the answer in single number, either 2 if leans slightly towards contradiction and 4 if it leans slightly towards entailment.
"""


Due to the existence of neutral prediction, we will further force LLM to classify whether the responses tend to leans on the entailment or contradiction side

In [ ]:
# Apply the neutral prompt for cases where the score is 3
def apply_neutrality_results(score_column, result_column, new_column_name):
    mask = df[score_column] == 3
    if mask.sum() == 0:
        print(f"✅ No neutral cases found for {score_column}. Skipping...")
        df[new_column_name] = None
        return df
    print(f"⚡ Processing {mask.sum()} neutral cases for {score_column}...")

    # Apply the neutral prompt where score == 3
    df.loc[mask, new_column_name] = df[mask].progress_apply(
        lambda row: get_entailment_decision(row['t1_japan'], row['t2_japan'], neutral_prompt_template), axis=1
    )

    return df

# Apply neutrality prompt where score == 3 for all methods
df = apply_neutrality_results('cot_score', 'cot_result', 'cot_neutral_result')
df = apply_neutrality_results('tot_score', 'tot_result', 'tot_neutral_result')
df = apply_neutrality_results('got_score', 'got_result', 'got_neutral_result')

print("🎉 All neutral cases processed and saved successfully!")


⚡ Processing 20 neutral cases for cot_score...


 10%|█         | 2/20 [00:08<01:12,  4.03s/it]

Legal Factors for Neutrality:
The neutrality in the case arises from the lack of clear information on whether the conditions A or B apply. The claim can be valid under Condition A where the guarantor's liability is restricted to the limit of the principal debt (under Article 448), and invalid under Condition B, where some other factor or exception may apply. A clear precedent that definitively resolves this dilemma is also absent, and thus the legitimacy of the claim hangs in balance until more specific particulars are assessed.

Japanese Legal Interpretation:
The case is primarily governed under Civil Code Article 448 and appears to be about a surety's liability reduction. However, in interpreting the rule, we consider a separate but critical legal issue: Whether the release of the main debtor affects the liability of a joint surety. While no explicit code addresses this exact scenario, similar cases in Japanese jurisprudence often consider the interrelationship between the main debto

 15%|█▌        | 3/20 [00:15<01:35,  5.62s/it]

Legal Factors for Neutrality:
The neutrality in this case stems from the application of Article 445 of the Japanese Civil Code, which describes the responsibility of sureties after one of them has been relieved of the joint liability. Here, if A released another party, C, from joint liability, the weight of C's share could revert back to the remaining sureties (B and D) if C has no financial capability to perform their part. Yet, without the specific condition that C is incapable of contributing, the distributive treatment between the remaining parties stays undetermined. This points toward the case being neutral, as it holds differing outcomes under various circumstances (Condition A, that C has no financial capability, supports Yes; while Condition B, that C or A has the capability, implies a No) pending the clarification on C's financial capacity.

Japanese Legal Interpretation:
The diplomatic rule that applies in this situation is Article 445, which sets forth the condition(s) unde

...

* Legal Factors for Neutrality: The neutrality in this case stems from the interplay between conditions A and B. Condition A aligns with the principle set forth in the legal article 564 and 565; under these provisions, the buyer would have grounds to seek redress or even to nullify the contract if the seller transfers rights that are not entirely their own, or if such rights do not conform to the conditions stipulated in the contract. However, condition B concerns the buyer’s prior knowledge concerning the rights being transferred at the time of the contract's execution. If Condition B is met (i.e., the buyer was aware that the seller did not wholly own the rights being transferred), it may bar the buyer from seeking the contract's termination or claiming compensation contingent on this fact alone. The neutrality comes from assessing whether the buyer's awareness of the specific rights not belonging to the seller fully justifies the loss of right to terminate the contract.
   
* Japane

100%|██████████| 22/22 [02:17<00:00,  7.47s/it]

* Legal Factors for Neutrality: In the given legal statement, we have two conditions that lead to opposing outcomes. If we look at 第五百六十一条 (Article 561), it imposes the liability on the seller to transfer rights to the buyer. On the other hand, 第四百十五条 (Article 415) states that the liability of the seller for the buyer's damages is waived when the failure to perform the duty is not due to the seller's fault. In this scenario, the claim might be neutral depending on whether Condition A (seller's failure to transfer rights) or Condition B (the failure not being the seller's fault) prevails.

* Japanese Legal Interpretation: The Civil Code often interprets that the seller is liable for the performance of obligations they promised. However, when the seller's incapacity to transfer rights is beyond reasonable control, there’s an established legal phrase in Article 415, "債務者の責めに帰することができない事由" (the fault cannot be attributed to the debtor). This clause aims to protect parties who refrain from b

100%|██████████| 22/22 [02:23<00:00,  6.51s/it]

Legal Factors for Neutrality: The neutrality of the case primarily derives from the conditions outlined by the legal statement which the validity of the claim depends on. The buyer's ability to claim compensation for damages is contingent on two main factors: 1) the buyer's awareness of the seller's inability to transfer a right belonging to a third party; and 2) whether or not the buyer notified the seller about the right belonging to a third party within a year of obtaining that knowledge.

Japanese Legal Interpretation: The mentioned legal article (Articles 566 of the Japanese Civil Code) dominates the right to seek remedies for breach of contract. This article sets specific conditions for when a buyer can make requests for remedy, reduction of the purchase price, claims for damages, and contract termination if the seller has delivered a product inconsistent with the contractual agreement. Although it primarily deals with defects in goods, the reasoning could apply analogically to t

In [ ]:
# Define a function to copy results if the score is not 3
def copy_results_if_not_neutral(score_column, result_column, neutral_column):
    # If score is not 3 and neutral result is missing, copy the original result
    df.loc[df[score_column] != 3, neutral_column] = df.loc[df[score_column] != 3, result_column]

# Copy results where score is not 3 for all methods
copy_results_if_not_neutral('cot_score', 'cot_result', 'cot_neutral_result')
copy_results_if_not_neutral('tot_score', 'tot_result', 'tot_neutral_result')
copy_results_if_not_neutral('got_score', 'got_result', 'got_neutral_result')

In [ ]:
# Apply the extraction function to each result column
df['cot_neutral_score'] = df['cot_neutral_result'].apply(extract_entailment_number)
df['tot_neutral_score'] = df['tot_neutral_result'].apply(extract_entailment_number)
df['got_neutral_score'] = df['got_neutral_result'].apply(extract_entailment_number)

Evaluation of the method

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

# Define a function to assign labels based on the score
def assign_label(score):
    if pd.isnull(score):  # Handle missing values
        return None
    elif score < 3:
        return 'N'
    elif score > 3:
        return 'Y'
    else:
        return None  # Exclude if score == 3

# Apply label assignment to each score column
df['cot_label'] = df['cot_neutral_score'].apply(assign_label)
df['tot_label'] = df['tot_neutral_score'].apply(assign_label)
df['got_label'] = df['got_neutral_score'].apply(assign_label)

# Apply label assignment to the average label
df['avg_label'] = df[['cot_label', 'tot_label', 'got_label']].apply(lambda x: 'Y' if x.tolist().count('Y') > x.tolist().count('N') else ('N' if x.tolist().count('N') > x.tolist().count('Y') else None), axis=1)

# Define a function to calculate F1 and accuracy for a given method
def calculate_metrics(predicted_labels, true_labels):
    valid_rows = ~pd.isnull(predicted_labels) & ~pd.isnull(true_labels)
    print(valid_rows.shape) # check for label = 3
    y_pred = predicted_labels[valid_rows]
    y_true = true_labels[valid_rows]

    if len(y_pred) == 0:  # Handle case where no valid predictions exist
        return {'f1': None, 'accuracy': None}

    f1 = f1_score(y_true, y_pred, pos_label='Y')
    acc = accuracy_score(y_true, y_pred)

    return {'f1': f1, 'accuracy': acc}

# Ground truth column (replace with the actual column name in your CSV)
df['label'] = df['label'].astype(str)

# Calculate metrics for each method
cot_metrics = calculate_metrics(df['cot_label'], df['label'])
tot_metrics = calculate_metrics(df['tot_label'], df['label'])
got_metrics = calculate_metrics(df['got_label'], df['label'])
avg_metrics = calculate_metrics(df['avg_label'], df['label'])

# Print the results
print("📊 Performance Metrics:")
print(f"CoT Method → F1 Score: {cot_metrics['f1']:.4f}, Accuracy: {cot_metrics['accuracy']:.4f}")
print(f"ToT Method → F1 Score: {tot_metrics['f1']:.4f}, Accuracy: {tot_metrics['accuracy']:.4f}")
print(f"GoT Method → F1 Score: {got_metrics['f1']:.4f}, Accuracy: {got_metrics['accuracy']:.4f}")
print(f"Average Score Method → F1 Score: {avg_metrics['f1']:.4f}, Accuracy: {avg_metrics['accuracy']:.4f}")

(216,)
(216,)
(216,)
(216,)
📊 Performance Metrics:
CoT Method → F1 Score: 0.8171, Accuracy: 0.7824
ToT Method → F1 Score: 0.8048, Accuracy: 0.7731
GoT Method → F1 Score: 0.8000, Accuracy: 0.7593
Average Score Method → F1 Score: 0.8300, Accuracy: 0.8009


In [ ]:
# Save the updated dataframe with results
result = df[['id','label','t1_japan','t2_japan','t1','t2','cot_label','tot_label','got_label','avg_label']]
result.to_csv('XXRunCoT_results.csv',  index=False, encoding='utf-8-sig')

print("Entailment analysis completed and saved to 'XXRunCoT_results.csv'")

The final result for H30-R02 predictions can be seen here

https://drive.google.com/file/d/1lySsID6bCWgSG6K_K9XD4-o-JNcppHPn/view?usp=sharing

## XXRunLang

In this section, we will demonstrate how our XXRunLang strategy works, we utilize prediction strategy for this

In [ ]:
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import os
import re
tqdm.pandas()

df = pd.read_csv('/content/COLIEE_dataset_H30-R02.csv')
# Create empty columns for results if they don't exist
df['prompt4_result'] = None
df['prompt5_result'] = None
df['prompt10_result'] = None

In [ ]:
# Define a function to extract "Yes" or "No" or "True" or "False" from the result
def extract_yes_no(response):
    if pd.isnull(response) or response == "Error" or response == "":
        return None  # Return None for missing or erroneous responses

    # Regex pattern to match variations
    pattern = re.compile(r'\b(Yes|True|False|No)\b', re.IGNORECASE)

    # Split response into words in reverse order
    words = response.split()[::-1]  # Reverse the response

    # Iterate over reversed words to find the first result
    for word in words:
        # Clean the word to remove extra characters
        clean_word = re.sub(r'[^a-zA-Z]', '', word)
        if pattern.match(clean_word):
            return clean_word.capitalize()

    return None  # Return None if no match is found

In this method, we utilize the Qwen 2-72B LLM from OpenRouter, you need to create your own API Key to try on this code

In [ ]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="<YOUR_API_KEY>",
)

In [ ]:
#@title Prompt Template for Language Analysis
# Define the prompts
prompt4_template = """
{t1}

Does this imply

{t2}

Please answer A) yes or B) no.

You can add reasoning first. But, at the end of your response, state the answer in single word, either yes or no.
"""

prompt5_template = """
{t1}

Does this mean that

{t2}

is true? A) yes or B) no.

You can add reasoning first. But, at the end of your response, state the answer in single word, either yes or no.
"""

prompt10_template = """
Take the following as truth:

{t1}

Then the following statement:

{t2}

is [true] or [false]?

You can add reasoning first. But, at the end of your response, state the answer in single word, either true or false.
"""



This code is dedicated to forward the prompted statements to the LLM. If there are entries that aren't evaluated, it will only re-iterate the entries that haven't been predicted

In [ ]:
# Define a function to get the response using OpenRouter API
def get_entailment_decision(article, statement, prompt_template):
    prompt = prompt_template.format(t1=article, t2=statement)
    try:
        completion = client.chat.completions.create(
            extra_headers={
                "HTTP-Referer": "<YOUR_SITE_URL>",
                "X-Title": "<YOUR_SITE_NAME>",
            },
            model="qwen/qwen-2-72b-instruct",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        print(completion.choices[0].message.content.strip())
        return completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error: {e}")
        return "Error"

# Function to select the correct t1 and t2 based on same_translate
def get_text(row):
    return row['t1'], row['t2']
    # return row['t1_japan'], row['t2_japan']

# Function to apply prompts and save intermediate results
def apply_prompt_results(result_column, prompt_template, prompt_name):
    """
    Apply prompt evaluation where results are NaN or 'Error' and save intermediate results.
    """
    mask = df[result_column].isnull() | (df[result_column] == "Error") | (df[result_column] == "")
    if mask.sum() == 0:
        print(f"✅ All rows for {prompt_name} are already processed. Skipping...")
        return df

    print(f"⚡ Processing {mask.sum()} rows for {prompt_name}...")

    # Apply the prompt where results are not available
    df.loc[mask, result_column] = df[mask].progress_apply(
        lambda row: get_entailment_decision(*get_text(row), prompt_template), axis=1
    )

    # Save intermediate results after processing
    df.to_csv('evaluated_results.csv', index=False)
    print(f"✅ Results for {prompt_name} saved to 'evaluated_results.csv'")
    return df

In [ ]:
# Prediction using Prompt 4
df = apply_prompt_results('prompt4_result', prompt4_template, "Prompt 4")

⚡ Processing 217 rows for Prompt 4...


  1%|          | 2/217 [00:02<05:01,  1.40s/it]

Based on the provided articles, when two or more movable properties with different owners are mixed to the point they can no longer be distinguished, the ownership of the composite thing belongs to the owner of the principal movable property. In this case, liquid R is a composite thing, with liquid P being the principal component due to its larger quantity. Therefore, A, as the owner of the principal component, acquires ownership of liquid R. Since liquid Q originally belonged to B and is now part of liquid R, B has suffered a loss. B can demand compensation from A based on the principles of unjust enrichment as outlined in Articles 703 and 704.

Answer: yes.


  1%|▏         | 3/217 [00:05<06:47,  1.90s/it]

The reasoning for the response is based on the interpretation of Article 244. If the distinction of principal and accessory cannot be made between the movables united by accession, co-ownership is established in proportion to the respective price current at the time of the accession. In this scenario, the engine owned by B is worth four million yen, and the ship owned by A is worth six million yen. Combined, they are worth ten million yen. The share of the ship with the engine would be in proportion to this valuation, resulting in a 3:2 ratio, with A having a 60% share (3 parts of 5) and B having a 40% share (2 parts of 5).

Therefore, the answer is yes.

yes


...

Based on Article 201 (1), A has a limited time to file an action for maintenance of possession. Since the construction on A's land started without consent, A could have filed an action during the construction or within one year after it stopped. Given that the fence was completed two years ago, A has missed the one-year deadline to file an action after the obstruction stopped. Therefore, as of this time, A may not demand the removal of the fence against B by filing an action for maintenance of possession.

Answer: yes.


100%|██████████| 217/217 [09:35<00:00,  2.27s/it]

The reasoning behind this question lies in understanding the rights of a possessor under the law. Article 200 (1) indicates that a possessor (in this context, B, who has possession of the machine tool via a lease agreement) has the right to demand restitution of the thing (the machine tool) and compensation for loss or damage if they are forcibly dispossessed (due to theft by C). This right exists independently of the owner (A) because B is recognized as the lawful possessor during the lease period.

Answer: yes.


100%|██████████| 217/217 [09:38<00:00,  2.66s/it]

The question as stated does not provide enough context to determine if the presence of "nan" implies another "nan". In general, "nan" stands for "not a number", typically representing an undefined or unrepresentable value in numerical computations. One instance of "nan" does not inherently imply the existence of another "nan"; it would depend on the specific context or scenario being considered.

Therefore, the answer is: no. However, without specific context, any answer could be speculative. In a different context, two "nan" values might indeed be related, but that is not implied by the general concept of "nan" itself. The answer here is based on the information provided, which is minimal.

Answer: no.
✅ Results for Prompt 4 saved to 'evaluated_results.csv'


In [ ]:
# Prediction using Prompt 5
df = apply_prompt_results('prompt5_result', prompt5_template, "Prompt 5")

⚡ Processing 126 rows for Prompt 5...


  2%|▏         | 2/126 [00:02<02:11,  1.06s/it]

The statement is true in certain circumstances under the principle outlined in Article 415. If the defect in the subject matter of work cannot be repaired, if the contractor refuses to repair it, or if the contract for the work has been cancelled due to the defect, then pursuant to Article 415(2), the party ordering the work may claim compensation for loss or damage in lieu of the performance (i.e., repair) of the obligation.

yes.


  2%|▏         | 3/126 [00:05<03:45,  1.83s/it]

The reasoning behind your interpretation is founded on the language of Article 572, which establishes that sellers cannot escape liability for warranty claims based on defects they knew about but did not disclose, regardless of any special agreements stating otherwise. Since the context you're applying this to is a contractor in a contract for value (which, according to Article 559, the provisions of warranty would be applicable to mutatis mutandis), it follows that even if the contractor attempts to set up a special agreement to avoid warranty liabilities, they would still be held accountable for defects they knew about and didn't disclose. Therefore, your interpretation is indeed accurate in this context, as the principle outlined in Article 572 transcends the specific instance of a seller and applies to contracts for value, encompassing the scenario of a contractor.

yes


...

Based on Article 201 (1), an action for maintenance of possession must be filed during the obstruction or within one year after the obstruction stops. Since B's construction of the fence on A's land was completed two years prior, and no action was taken by A within the one-year timeframe after the obstruction stopped, A is not within the legal timeframe to file an action for maintenance of possession against B. Therefore, as of the time the fence was completed, A may not demand the removal of the fence from his land by filing an action for maintenance of possession.

Answer: yes.


100%|██████████| 126/126 [05:40<00:00,  2.31s/it]

The statement is true, based on Article 200 (1). This provision grants a possessor, in this case B, the right to demand restitution of the thing they possess, regardless of whether they are the owner. Therefore, B, as the possessor under the lease agreement, has the right to file an action for recovery of possession against C, independent of A's actions.

yes.


100%|██████████| 126/126 [05:41<00:00,  2.71s/it]

In computing, "NaN" stands for "Not a Number," which is a special floating-point value that indicates an undefined or unrepresentable value in calculations. Therefore, the statement "nan" cannot be true or false in a boolean context; it is a value that falls outside the realm of typical true/false or numerical value assessments.

no
✅ Results for Prompt 5 saved to 'evaluated_results.csv'


In [ ]:
# Prediction using Prompt 10
df = apply_prompt_results('prompt10_result', prompt10_template, "Prompt 10")

⚡ Processing 217 rows for Prompt 10...


  1%|          | 2/217 [00:02<05:21,  1.50s/it]

The statement is true based on the legal principles provided in the articles. Article 243 asserts that if different owner's movable properties can no longer be separated without damage due to accession, the ownership of the composite thing belongs to the owner of the principal movable property. In this case, liquid P owned by A is the principal movable property due to its larger volume, thus A acquires ownership of the composite liquid R. Article 248 allows for a person that suffers loss because of the application of the provisions of Article 243 to demand compensation. Furthermore, Articles 703 and 704 allow for compensation in cases of unjust enrichment, which would apply if B suffered a loss due to the blending of the liquids.

True.


  1%|▏         | 3/217 [00:05<07:01,  1.97s/it]

Reasoning: According to Article 244, when movables are united by accession and the distinction between principal and accessory cannot be made, co-ownership occurs in proportion to their respective prices at the time of the accession. In this case, B's engine is worth four million yen, and A's ship is worth six million yen. Together, they are worth ten million yen. The ratio of their contributions is 4:6, which simplifies to 2:3. However, the statement inverses this ratio, indicating that A and B shall share the ship with the engine in a ratio of three to two, which is consistent with the principle of co-ownership in proportion to the respective prices.

Answer: True.


...

Reasoning: According to Article 201 (1), the action for maintenance of possession must be filed during the obstruction or within one year after the obstruction stops. Since B's obstruction (construction of the fence) on A's land has been completed and more than one year has passed since the construction started, A cannot file an action for maintenance of possession against B as per the stipulated conditions of the article.

Answer: True.


100%|██████████| 217/217 [08:43<00:00,  2.40s/it]

Reasoning: According to Article 200 (1), a possessor has the right to demand the restitution of a thing and compensation for loss or damage if they are forcibly dispossessed. In the given scenario, B is the possessor of the machine tool because B has leased it from A. When C steals the machine tool from B, B is forcibly dispossessed. B would have a right under the law to demand the restitution of the thing and compensation for loss from C, as B is the one directly affected by the theft. This does not necessarily require A's involvement, as B's legal right to possess the machine tool is provided through the lease agreement.

Answer: True


100%|██████████| 217/217 [08:47<00:00,  2.43s/it]

The statement provided is "nan" which stands for "not a number" in mathematical and computing contexts. This term is used to represent values that cannot be defined or are not within the range of possible numbers. Since you have asked to take "nan" as truth, it implies that we are considering "nan" to be a valid or true input in the current context. However, when you ask if the following identical statement ("nan") is true or false, it's important to recognize that "nan" does not represent a boolean value and cannot itself be true or false in a boolean sense—it is neither true nor false; it's a separate category indicating an indeterminate numerical value.

Thus, the question of whether "nan" is true or false is unanswerable in terms of classical logic, as "nan" does not fit into the binary classification of true or false. However, per your instruction to provide a single-word answer based on the interpretation that "nan" has been established as a truth in this particular context, the 

We utilize regex to extract the response from the LLM, then evaluate the model

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
import re

# Define a function to extract "Yes" or "No" from the result
def extract_yes_no(response):
    if pd.isnull(response) or response == "Error" or response == "":
        return None  # Return None for missing or erroneous responses

    # Regex pattern to match variations of "Yes" and "No"
    pattern = re.compile(r'\b(Yes|True|False|No)\b', re.IGNORECASE)

    # Split response into words in reverse order
    words = response.split()[::-1]  # Reverse the response

    # Iterate over reversed words to find the first "Yes" or "No"
    for word in words:
        # Clean the word to remove extra characters
        clean_word = re.sub(r'[^a-zA-Z]', '', word)
        if pattern.match(clean_word):
            return clean_word.capitalize()  # Return "Yes" or "No" in proper case

    return None  # Return None if no match is found

# Apply the extraction function to each result column
print("⚡ Extracting results from prompt responses...")
df['prompt4_label'] = df['prompt4_result'].apply(extract_yes_no)
df['prompt5_label'] = df['prompt5_result'].apply(extract_yes_no)
df['prompt10_label'] = df['prompt10_result'].apply(extract_yes_no)

# Convert Yes/No to Y/N for evaluation
def convert_to_binary(label):
    if label.lower() == 'yes' or label.lower() == 'true':
        return 'Y'
    elif label.lower() == 'no' or label.lower() == 'false':
        return 'N'
    else:
        return None

# Convert extracted labels to binary
df['prompt4_binary'] = df['prompt4_label'].apply(convert_to_binary)
df['prompt5_binary'] = df['prompt5_label'].apply(convert_to_binary)
df['prompt10_binary'] = df['prompt10_label'].apply(convert_to_binary)

# Define a function to calculate F1 score and accuracy
def calculate_metrics(predicted_labels, true_labels):
    # Filter out rows where the predictions are None
    valid_rows = ~pd.isnull(predicted_labels) & ~pd.isnull(true_labels)
    y_pred = predicted_labels[valid_rows]
    y_true = true_labels[valid_rows]

    if len(y_pred) == 0:
        return {'f1': None, 'accuracy': None}

    f1 = f1_score(y_true, y_pred, pos_label='Y')
    acc = accuracy_score(y_true, y_pred)

    return {'f1': f1, 'accuracy': acc}

# Assume the ground truth labels are in the 'label' column
df['label'] = df['label'].astype(str)

# Calculate metrics for each prompt
prompt4_metrics = calculate_metrics(df['prompt4_binary'], df['label'])
prompt5_metrics = calculate_metrics(df['prompt5_binary'], df['label'])
prompt10_metrics = calculate_metrics(df['prompt10_binary'], df['label'])

# Print the evaluation results
print("📊 Evaluation Results:")
print(f"Prompt 4 → F1 Score: {prompt4_metrics['f1']:.4f}, Accuracy: {prompt4_metrics['accuracy']:.4f}")
print(f"Prompt 5 → F1 Score: {prompt5_metrics['f1']:.4f}, Accuracy: {prompt5_metrics['accuracy']:.4f}")
print(f"Prompt 10 → F1 Score: {prompt10_metrics['f1']:.4f}, Accuracy: {prompt10_metrics['accuracy']:.4f}")

⚡ Extracting results from prompt responses...
📊 Evaluation Results:
Prompt 4 → F1 Score: 0.8072, Accuracy: 0.8009
Prompt 5 → F1 Score: 0.8267, Accuracy: 0.8194
Prompt 10 → F1 Score: 0.8120, Accuracy: 0.7963


From those three predicions, we employ a voting mechanism and evaluate it

In [ ]:
# Function to apply majority voting for predictions
def apply_majority_vote(row):
    votes = [row['prompt4_binary'], row['prompt5_binary'], row['prompt10_binary']]
    if votes.count('Y') > 1:
        return 'Y'
    elif votes.count('N') > 1:
        return 'N'
    else:
        return None  # If there's a tie or no valid vote, return None

# Apply majority voting across the rows
df['final_prediction'] = df.apply(apply_majority_vote, axis=1)

# Calculate metrics for the final prediction
final_metrics = calculate_metrics(df['final_prediction'], df['label'])

# Print the evaluation results
print("📊 Evaluation Results:")
print(f"Final Prediction → F1 Score: {final_metrics['f1']:.4f}, Accuracy: {final_metrics['accuracy']:.4f}")

📊 Evaluation Results:
Final Prediction → F1 Score: 0.8370, Accuracy: 0.8287


In [ ]:
# Save the updated dataframe with labels and binary results
result = df[['id','label','t1_japan','t2_japan','t1','t2','prompt4_binary','prompt5_binary','prompt10_binary','final_prediction']]
result.to_csv('XXRunLang_results.csv',  index=False, encoding='utf-8-sig')
print("✅ Evaluation complete. Results saved to 'XXRunLang_results.csv'")

The prediction result for the XXRunLang can be accessed here

https://drive.google.com/file/d/18QXC5P_1AryOv6UYdzoOlyVeF1KQQWgS/view?usp=sharing